In [ ]:
#Import dependencies
import pandas as pd
import requests
from statsmodels.stats.weightstats import ttest_ind
!pip install covid19dh --upgrade > nul
!pip install pingouin  --upgrade > nul
import matplotlib # visualization lib
import matplotlib.pyplot as plt
import numpy as np
import datetime
import seaborn as sns
import plotly.express as px
import plotly
import plotly.graph_objects as go
from scipy.stats import ttest_ind
from plotly.subplots import make_subplots
from pathlib import Path
%matplotlib inline

In [ ]:
# Get datasource
from covid19dh import covid19

# Get csv data for after vaccine data
file = Path(r'C:\Users\Chloe\OneDrive\Desktop\Bootcamp2\AllHomework\Resource\usa_coviddata_2022.csv')


In [ ]:
# use URL to pull out data
# url = "https://storage.covid19datahub.io/level/2.csv"

# # download the file from the URL using requests
# response = requests.get(url)
# content = response.content

# # create a pandas dataframe from the downloaded content
# file = pd.read_csv(content.decode('utf-8'))


In [ ]:
#Return data for states in USA
states_df,states_src = covid19("USA", level = 2, verbose = False)

#Filter before and after vaccine dataframes
statesBV_df,statesbeforevaccine_src = covid19("USA", level = 2, start = datetime.date(2020,1,1), end = datetime.date(2020,12,13), verbose = False)
statesAV_df = pd.read_csv(file, encoding="ISO-8859-1")


#statesAV_df,statesaftervaccine_src = covid19("USA", level = 2, start = datetime.date(2020,12,14), end = datetime.date(2023,4,15), verbose = False)


In [ ]:
states_df

In [ ]:
#Display unique states
states_df.key_apple_mobility.unique()

In [ ]:
#Display columns
states_df.columns

In [ ]:
# Remove territories of United States
states_df = states_df.loc[(states_df["key_apple_mobility"] != "Northern Mariana Islands")]
states_df = states_df.loc[(states_df["key_apple_mobility"] != "Guam")]
states_df = states_df.loc[(states_df["key_apple_mobility"] != "American Samoa")]
states_df = states_df.loc[(states_df["key_apple_mobility"] != "Puerto Rico")]
states_df

# Remove territories of United States for Before Vaccine df
statesBV_df = statesBV_df.loc[(statesBV_df["key_apple_mobility"] != "Northern Mariana Islands")]
statesBV_df = statesBV_df.loc[(statesBV_df["key_apple_mobility"] != "Guam")]
statesBV_df = statesBV_df.loc[(statesBV_df["key_apple_mobility"] != "American Samoa")]
statesBV_df = statesBV_df.loc[(statesBV_df["key_apple_mobility"] != "Puerto Rico")]
statesBV_df

# Remove territories of United States for After Vaccine df
statesAV_df = statesAV_df.loc[(statesAV_df["key_apple_mobility"] != "Northern Mariana Islands")]
statesAV_df = statesAV_df.loc[(statesAV_df["key_apple_mobility"] != "Guam")]
statesAV_df = statesAV_df.loc[(statesAV_df["key_apple_mobility"] != "American Samoa")]
statesAV_df = statesAV_df.loc[(statesAV_df["key_apple_mobility"] != "Puerto Rico")]
statesAV_df

In [ ]:
#Confirm that territories were removed
states_df.key_apple_mobility.unique()

In [ ]:
#Create dictionary for state abbreviations
us_state_abbrev = {
'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}

In [ ]:
# Create a new column that finds the Confirmed % for each date. 
states_df["confirmed_percentage"] = states_df["confirmed"] / states_df["population"]
statesBV_df["confirmed_percentage"] = statesBV_df["confirmed"] / statesBV_df["population"]
statesAV_df["confirmed_percentage"] = statesAV_df["confirmed"] / statesAV_df["population"]


# Create a new column that finds the Vaccination % for each date. 
states_df["vaccinated_percentage"] = states_df["people_vaccinated"] / states_df["population"]
statesBV_df["vaccinated_percentage"] = statesBV_df["people_vaccinated"] / statesBV_df["population"]
statesAV_df["vaccinated_percentage"] = statesAV_df["people_vaccinated"] / statesAV_df["population"]


# Create a new column that finds the Recovered % for each date. 
states_df["recovered_percentage"] = states_df["recovered"] / states_df["population"]
statesBV_df["recovered_percentage"] = statesBV_df["recovered"] / statesBV_df["population"]
statesAV_df["recovered_percentage"] = statesAV_df["recovered"] / statesAV_df["population"]


# Create a new column that finds the Death Rate % for each date. 
states_df["deathrate_percentage"] = states_df["deaths"] / states_df["population"]
statesBV_df["deathrate_percentage"] = statesBV_df["deaths"] / statesBV_df["population"]
statesAV_df["deathrate_percentage"] = statesAV_df["deaths"] / statesAV_df["population"]



# Create a new column that determines the change rate of confirmed percentages
states_df["growth_rate"] = states_df['confirmed_percentage'].pct_change(periods=1) * 100
statesBV_df["growth_rate"] = statesBV_df['confirmed_percentage'].pct_change(periods=1) * 100
statesAV_df["growth_rate"] = statesAV_df['confirmed_percentage'].pct_change(periods=1) * 100


# Create a new column that determines the state abbreviation
states_df["state_abbreviation"] = states_df['key_apple_mobility'].map(us_state_abbrev)
statesBV_df["state_abbreviation"] = statesBV_df['key_apple_mobility'].map(us_state_abbrev)
statesAV_df["state_abbreviation"] = statesAV_df['key_apple_mobility'].map(us_state_abbrev)



states_df["confirmed_percentage"]
states_df["vaccinated_percentage"]
states_df["recovered_percentage"]
states_df["deathrate_percentage"]
states_df["growth_rate"]
states_df["state_abbreviation"]


In [ ]:
#Rename columns

states_df = states_df.loc[:,["administrative_area_level_1", "key_apple_mobility", "state_abbreviation", "date", "confirmed",
                             "people_vaccinated", "people_fully_vaccinated", "population","confirmed_percentage", 
                             "vaccinated_percentage", "recovered_percentage", "deathrate_percentage", "growth_rate"]]
states_df.rename(columns = {"administrative_area_level_1": "Country",
                            "key_apple_mobility": "State",
                            "state_abbreviation": "State Abbreviation",
                            "date": "Date", 
                            "confirmed": "Confirmed Cases",
                            "people_vaccinated": "Vaccinated",
                            "people_fully_vaccinated": "Fully Vaccinated",
                            "population": "Population",
                            "confirmed_percentage": "Confirmed Percentage by Population",
                            "vaccinated_percentage": "Vaccinated Percentage by Population",
                            "recovered_percentage": "Recovered Percentage by Population",
                            "deathrate_percentage": "Death Rate Percentage by Population",
                            "growth_rate": "Growth Rate"},
                            inplace = True)

#Rename columns for Before Vaccine df
statesBV_df = statesBV_df.loc[:,["administrative_area_level_1", "key_apple_mobility", "state_abbreviation", "date", "confirmed",
                                 "people_vaccinated", "people_fully_vaccinated", "population","confirmed_percentage", 
                                 "vaccinated_percentage", "recovered_percentage", "deathrate_percentage", "growth_rate"]]
statesBV_df.rename(columns = {"administrative_area_level_1": "Country",
                            "key_apple_mobility": "State",
                            "state_abbreviation": "State Abbreviation",
                            "date": "Date", 
                            "confirmed": "Confirmed Cases",
                            "people_vaccinated": "Vaccinated",
                            "people_fully_vaccinated": "Fully Vaccinated",
                            "population": "Population",
                            "confirmed_percentage": "Confirmed Percentage by Population",
                            "vaccinated_percentage": "Vaccinated Percentage by Population",
                            "recovered_percentage": "Recovered Percentage by Population",
                            "deathrate_percentage": "Death Rate Percentage by Population",
                            "growth_rate": "Growth Rate"},
                            inplace = True)

#Rename columns for After Vaccine df
statesAV_df = statesAV_df.loc[:,["administrative_area_level_1", "key_apple_mobility", "state_abbreviation", "date", "confirmed",
                                 "people_vaccinated", "people_fully_vaccinated", "population","confirmed_percentage", 
                                 "vaccinated_percentage", "recovered_percentage", "deathrate_percentage", "growth_rate"]]
statesAV_df.rename(columns = {"administrative_area_level_1": "Country",
                            "key_apple_mobility": "State",
                            "state_abbreviation": "State Abbreviation",
                            "date": "Date", 
                            "confirmed": "Confirmed Cases",
                            "people_vaccinated": "Vaccinated",
                            "people_fully_vaccinated": "Fully Vaccinated",
                            "population": "Population",
                            "confirmed_percentage": "Confirmed Percentage by Population",
                            "vaccinated_percentage": "Vaccinated Percentage by Population",
                            "recovered_percentage": "Recovered Percentage by Population",
                            "deathrate_percentage": "Death Rate Percentage by Population",
                            "growth_rate": "Growth Rate"},
                            inplace = True)

states_df

In [ ]:
# Show highly affected areas of United States

fig = px.choropleth(states_df,
                    locations='State Abbreviation', 
                    locationmode="USA-states", 
                    scope="usa",
                    color='Confirmed Cases',
                    color_continuous_scale='Viridis_r', 
                    
                    )
fig.show()

In [ ]:
# Top 5 Confirmed % by Population (h-Bar plot)
top5_confirmed = pd.DataFrame(states_df.groupby('State')['Confirmed Percentage by Population'].max().nlargest(5).sort_values(ascending = True))
fig2 = px.bar(top5_confirmed, x = 'Confirmed Percentage by Population', y = top5_confirmed.index, height = 300, color = 'Confirmed Percentage by Population', orientation = 'h',
            color_continuous_scale = ['deepskyblue','red'], title = 'Most Affected States in United States') 
fig2.update_layout(
          hovermode="x",showlegend=False, autosize=True, xaxis=dict(tickformat=".1%"))
fig2.show()


# Top 5 Confirmed numbers (h-Bar plot) Before Vaccine
top5BV_confirmed = pd.DataFrame(statesBV_df.groupby('State')['Confirmed Percentage by Population'].max().nlargest(5).sort_values(ascending = True))
fig2 = px.bar(top5BV_confirmed, x = 'Confirmed Percentage by Population', y = top5BV_confirmed.index, height = 300, color = 'Confirmed Percentage by Population', orientation = 'h',
            color_continuous_scale = ['deepskyblue','red'], title = 'Most Affected States in United States Before Vaccine')
fig2.update_layout(
        hovermode="x",showlegend=False, autosize=True, xaxis=dict(tickformat=".1%"))
fig2.show()


# Top 5 Confirmed numbers (h-Bar plot) After Vaccine
top5AV_confirmed = pd.DataFrame(statesAV_df.groupby('State')['Confirmed Percentage by Population'].max().nlargest(5).sort_values(ascending = True))
fig2 = px.bar(top5AV_confirmed, x = 'Confirmed Percentage by Population', y = top5AV_confirmed.index, height = 300, color = 'Confirmed Percentage by Population', orientation = 'h',
            color_continuous_scale = ['deepskyblue','red'], title = 'Most Affected States in United States After Vaccine')
fig2.update_layout(
        hovermode="x",showlegend=False, autosize=True, xaxis=dict(tickformat=".1%"))
fig2.show()

In [ ]:
# Top 5 Vaccinated % by Population (h-Bar plot)
top5_vaccinated = pd.DataFrame(states_df.groupby('State')['Vaccinated Percentage by Population'].max().nlargest(5).sort_values(ascending = True))
fig2 = px.bar(top5_vaccinated, x = 'Vaccinated Percentage by Population', y = top5_vaccinated.index, height = 300, color = 'Vaccinated Percentage by Population', orientation = 'h',
            color_continuous_scale = ['gray','green'], title = 'Top 5 Vaccinated States in United States')
fig2.update_layout(
        hovermode="x",showlegend=False, autosize=True, xaxis=dict(tickformat=".1%"))
fig2.show()

top5_vaccinated = pd.DataFrame(statesAV_df.groupby('State')['Vaccinated Percentage by Population'].max().nlargest(5).sort_values(ascending = True))
fig2 = px.bar(top5_vaccinated, x = 'Vaccinated Percentage by Population', y = top5_vaccinated.index, height = 300, color = 'Vaccinated Percentage by Population', orientation = 'h',
            color_continuous_scale = ['gray','green'], title = 'Top 5 Vaccinated States in United States')
fig2.update_layout(
        hovermode="x",showlegend=False, autosize=True, xaxis=dict(tickformat=".1%"))
fig2.show()



In [ ]:
# Top 5 Recovered % by Population (h-Bar plot)
top5_recovered = pd.DataFrame(states_df.groupby('State')['Recovered Percentage by Population'].max().nlargest(5).sort_values(ascending = True))
fig2 = px.bar(top5_recovered, x = 'Recovered Percentage by Population', y = top5_recovered.index, height = 300, color = 'Recovered Percentage by Population', orientation = 'h',
            color_continuous_scale = ['gray','green'], title = 'Top 5 Recovered % States in United States')
fig2.update_layout(
        hovermode="x",showlegend=False, autosize=True, xaxis=dict(tickformat=".1%"))
fig2.show()


# Top 5 Recovered % by Population (h-Bar plot) Before Vaccine
top5BV_recovered = pd.DataFrame(statesBV_df.groupby('State')['Recovered Percentage by Population'].max().nlargest(5).sort_values(ascending = True))
fig2 = px.bar(top5BV_recovered, x = 'Recovered Percentage by Population', y = top5BV_recovered.index, height = 300, color = 'Recovered Percentage by Population', orientation = 'h',
            color_continuous_scale = ['gray','green'], title = 'Top 5 Recovered % in United States Before Vaccine')
fig2.update_layout(
        hovermode="x",showlegend=False, autosize=True, xaxis=dict(tickformat=".1%"))
fig2.show()


# Top 5 Recovered % by Population (h-Bar plot) After Vaccine
top5AV_recovered = pd.DataFrame(statesAV_df.groupby('State')['Recovered Percentage by Population'].max().nlargest(5).sort_values(ascending = True))
fig2 = px.bar(top5AV_recovered, x = 'Recovered Percentage by Population', y = top5AV_recovered.index, height = 300, color = 'Recovered Percentage by Population', orientation = 'h',
            color_continuous_scale = ['gray','green'], title = 'Top 5 Recovered % in United States After Vaccine')
fig2.update_layout(
        hovermode="x",showlegend=False, autosize=True, xaxis=dict(tickformat=".1%"))
fig2.show()

In [ ]:
# Top 5 Death Rate % by Population (h-Bar plot)
top5_deathrate = pd.DataFrame(states_df.groupby('State')['Death Rate Percentage by Population'].max().nlargest(5).sort_values(ascending = True))
fig2 = px.bar(top5_deathrate, x = 'Death Rate Percentage by Population', y = top5_deathrate.index, height = 300, color = 'Death Rate Percentage by Population', orientation = 'h',
            color_continuous_scale = ['deepskyblue','red'], title = 'Top 5 Death Rate % States in United States')
fig2.update_layout(
        hovermode="x",showlegend=False, autosize=True, xaxis=dict(tickformat=".1%"))
fig2.show()


# Top 5 Death Rate % by Population (h-Bar plot) Before Vaccine
top5BV_deathrate = pd.DataFrame(statesBV_df.groupby('State')['Death Rate Percentage by Population'].max().nlargest(5).sort_values(ascending = True))
fig2 = px.bar(top5BV_deathrate, x = 'Death Rate Percentage by Population', y = top5BV_deathrate.index, height = 300, color = 'Death Rate Percentage by Population', orientation = 'h',
            color_continuous_scale = ['deepskyblue','red'], title = 'Top 5 Death Rate % in United States Before Vaccine')
fig2.update_layout(
        hovermode="x",showlegend=False, autosize=True, xaxis=dict(tickformat=".1%"))
fig2.show()


# Top 5 Death Rate % by Population (h-Bar plot) After Vaccine
top5AV_deathrate = pd.DataFrame(statesAV_df.groupby('State')['Death Rate Percentage by Population'].max().nlargest(5).sort_values(ascending = True))
fig2 = px.bar(top5AV_deathrate, x = 'Death Rate Percentage by Population', y = top5AV_deathrate.index, height = 300, color = 'Death Rate Percentage by Population', orientation = 'h',
            color_continuous_scale = ['deepskyblue','red'], title = 'Top 5 Death Rate % in United States After Vaccine')
fig2.update_layout(
        hovermode="x",showlegend=False, autosize=True, xaxis=dict(tickformat=".1%"))
fig2.show()

In [ ]:
print(statesBV_df.isnull().sum())
print(statesAV_df.isnull().sum())

In [ ]:
statesBV_df.fillna(statesBV_df.mean(), inplace=True)
statesAV_df.fillna(statesAV_df.mean(), inplace=True)

In [ ]:
#HO:there is no significant difference in the death rates before and after vaccination 
#Ha:there is a significant difference.
# Define the data
death_rate_before = statesBV_df["Death Rate Percentage by Population"].values
death_rate_after = statesAV_df["Death Rate Percentage by Population"].values

# Calculate the means and standard deviations
mean_before = np.mean(death_rate_before)
mean_after = np.mean(death_rate_after)
std_before = np.std(death_rate_before, ddof=1)
std_after = np.std(death_rate_after, ddof=1)

# Calculate the t-value and p-value using a two-sample t-test
t, p = ttest_ind(death_rate_before, death_rate_after, equal_var=False)

# Print the results
print("Mean before vaccination:", mean_before)
print("Mean after vaccination:", mean_after)
print("Standard deviation before vaccination:", std_before)
print("Standard deviation after vaccination:", std_after)
print("t-value:", t)
print("p-value:", p)

In [ ]:
# Plot a box plot of the two groups
plt.boxplot([death_rate_before, death_rate_after], labels=['Before', 'After'])
plt.title('Death Rates Before and After Vaccination')
plt.ylabel('Death Rate')
plt.show()

In [ ]:
# Define the data
means = [mean_before, mean_after]
labels = ['Before', 'After']

# Plot the bar chart
plt.bar(labels, means)
plt.title('Mean Death Rates Before and After Vaccination')
plt.xlabel('Vaccination Status')
plt.ylabel('Mean Death Rate')
plt.show()

In [ ]:
#Conclusion: the death rate after vaccination is significantly higher than the death rate before vaccination
#Possible reasons: 
#1. vaccine is not as effective as we expected and vaccinated individuals are still at risk of dying from COVID-19.
#2. new variants of the virus may be contributing to the increased death rate after vaccination.
#Going Forward:
#It's important to note that there are lots of factors could affect the death rate, such as health conditions, illness, economic support, eating habits etc. So a fully understanding of the T-test results is needed.  